In [5]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

# Analyzing CIA Factbook Data Using SQL

In this guided project, we will study some interesting facts about our world using SQL.

Below is our data set `facts`.

## Understanding the data set

In [8]:
%%sql
SELECT *
FROM facts
LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [9]:
%%sql
SELECT min(population), max(population), min(population_growth), max(population_growth)
FROM facts

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,7256490011,0.0,4.02


Countries with minimum population

In [10]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT min(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


Countries with maximum population

In [11]:
%%sql
SELECT *
FROM facts
WHERE population = (SELECT max(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The data includes the data of the whole world, which is a summary for all countries. Therefore we will exclude it.

Recompute the summary statistics excluding the row for the whole world. Include the following:

- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [12]:
%%sql
SELECT min(population), max(population), min(population_growth), max(population_growth)
FROM facts
WHERE population != (SELECT max(population) FROM facts)

Done.


min(population),max(population),min(population_growth),max(population_growth)
0,1367485388,0.0,4.02


Average of population and area excluding the whole world data:

In [14]:
%%sql
SELECT avg(population), avg(area)
FROM facts
WHERE population != (SELECT max(population) FROM facts)

Done.


avg(population),avg(area)
32242666.56846473,582949.8523206752


## Densely populated countries, which have the following:

- Above-average values for population.
- Below-average values for area.

In [21]:
%%sql
SELECT name, area, population
FROM facts
WHERE (population > (SELECT avg(population) 
                    FROM facts
                    WHERE population != (
                        SELECT max(population) FROM facts
                    )
                   )) & (area < (SELECT avg(area) 
                    FROM facts
                    WHERE population != (
                        SELECT max(population) FROM facts
                    )
                   ))

Done.


name,area,population
Bangladesh,148460,168957745
Germany,357022,80854408
Iraq,438317,37056169
Italy,301340,61855120
Japan,377915,126919659
Kenya,580367,45925301
"Korea, South",99720,49115196
Morocco,446550,33322699
Philippines,300000,100998376
Poland,312685,38562189


## Country with the most people - China

In [27]:
%%sql
SELECT name, population
FROM facts
WHERE population == (SELECT max(population) 
                     FROM (SELECT * FROM facts WHERE name != 'World'))

Done.


name,population
China,1367485388


## Country with the highes growth rate - South Sudan

In [30]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth == (SELECT max(population_growth) 
                            FROM (SELECT * FROM facts WHERE name != 'World'))

Done.


name,population_growth
South Sudan,4.02


## Country with highest ratios of water to land - British Indian Ocean Territory

In [31]:
%%sql
SELECT name, area_water/area_land as water_land_ratio
FROM facts
WHERE cast(area_water as FLOAT)/area_land == (
    SELECT max(cast(area_water as FLOAT)/area_land) 
    FROM (SELECT * FROM facts WHERE name != 'World'))

Done.


name,water_land_ratio
British Indian Ocean Territory,905


## Countries with more water than land

In [32]:
%%sql
SELECT name, area_water, area_land
FROM (SELECT * FROM facts WHERE name != 'World')
WHERE area_water > area_land

Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


## Countries with higher death rate than birth rate

In [34]:
%%sql
SELECT name, death_rate, birth_rate
FROM (SELECT * FROM facts WHERE name != 'World')
WHERE death_rate > birth_rate

Done.


name,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45
Czech Republic,10.34,9.63
Estonia,12.4,10.51
Germany,11.42,8.47
Greece,11.09,8.66
Hungary,12.73,9.16
